# DCF companies to find deep value

## Params

In [1]:
import math

discount_rate = 0.12
years = 50
maturity = 15 #math.floor(years/2)
discount_threshold = 0.4
ddm_discount_threshold = 0.1
risk_free_rate = 0.03
mkt_return = .08

## Searching for undervalued companies

In [2]:
import json

# read file
with open('yahoo_tickers.json', 'r') as f:
    data=f.read()

# parse file
y_tickers_data = json.loads(data)
y_tickers = list(map(lambda x: x["symbol"], y_tickers_data["quotes"]))

In [3]:
import yfinance as yf
import pandas as pd
import numpy_financial as npf

def compute_dcf(t):
    print(t)
    try:
        ticker = yf.Ticker(t)
        df = ticker.history(period='1D', interval="1d")

        if len(df) == 0 or "sharesOutstanding" not in ticker.info or ticker.info["sharesOutstanding"] == None:
            return 1, 1, 1

        price = df["Close"][0]
        shares = ticker.info["sharesOutstanding"]
        cash = ticker.balance_sheet.T["Cash"].iloc[0]
        if math.isnan(cash):
            cash = ticker.balance_sheet.T["Cash"].iloc[1]

        netincome_df = ticker.cashflow.T[["Net Income"]]
        netincome_df = netincome_df[netincome_df["Net Income"] > 0]
        netincome_df = netincome_df.sort_values("")
        netincome_df = netincome_df.reset_index()
        
        netincome_df["Growth"] = (netincome_df["Net Income"] - netincome_df["Net Income"].shift())/netincome_df["Net Income"]
        netincome_df = netincome_df.fillna(0).iloc[1:, :][["Net Income", "Growth"]]

        growth = netincome_df["Growth"].mean(axis=0)
        if growth >= 0.3:
            growth = 0.3
        growth_slowdown = growth/2

        # forcast
        for i in range(years):
            row ={"Net Income": netincome_df["Net Income"].iloc[-1] * (growth+1), "Growth": growth}
            rowdf = pd.DataFrame.from_records([row])
            netincome_df = pd.concat([netincome_df, rowdf], ignore_index=True, axis=0)
            
            if i == maturity:
                growth = -growth_slowdown

        netincome_df = netincome_df.iloc[3:, :]
        cashflows = netincome_df["Net Income"].to_list()

        npv = npf.npv(discount_rate, cashflows);
        value = npv + cash
        forcast_price = value/shares

        # DDM
        ddm = 0
        if "dividendRate" in ticker.info and ticker.info["dividendRate"] != None and "beta" in ticker.info:
            exp_future_div = ticker.info["dividendRate"]
            div_growth = 0.03
            MKT_Risk_prem = mkt_return - risk_free_rate
            beta = ticker.info["beta"]
            COE = round(beta * MKT_Risk_prem + risk_free_rate,4)
            ddm = round(exp_future_div/(COE-div_growth),2)
        return forcast_price, price, ddm
        
    except:
        print("skipping {}".format(t))
        return 1, 1, 1



In [4]:
undervalued = []
undervalued_ddm = []

for t in y_tickers:
    fp, p, d = compute_dcf(t)
    if (fp - p)/p > discount_threshold:
        undervalued.append({"ticker": t, "forcast_price": fp, "current_price": p, "DDM": d})
    if d > p * (1+ddm_discount_threshold):
        undervalued_ddm.append({"ticker": t, "forcast_price": fp, "current_price": p, "DDM": d})

APC.DE
TL0.DE
MC.PA
SAP.DE
OR.PA
UNA.AS
PRX.AS
ASML.AS
SAN.PA
LIN.DE
SIE.DE
ABI.BR
RDSA.AS
- RDSA.AS: No data found, symbol may be delisted
R6C3.DE
- R6C3.DE: No data found, symbol may be delisted
R6C.DE
- R6C.DE: No data found, symbol may be delisted
RDSB.AS
- RDSB.AS: No data found, symbol may be delisted
FP.PA
- FP.PA: No data found, symbol may be delisted
ENEL.MI
VOW.DE
RMS.PA
VOW3.DE
ALV.DE
DTE.DE
ITX.MC
KER.PA
HBC1.DE
AI.PA
IBE.MC
3CP.F
BPE5.DE
SU.PA
BPE5.F
BAYN.DE
AIR.PA
AIR.DE
ADS.DE
MRK.DE
BAS.DE
DAI.DE
- DAI.DE: No data found, symbol may be delisted
EL.PA
DPW.DE
GAZ.DE
- GAZ.DE: No data found for this date range, symbol may be delisted
GAZ.F
- GAZ.F: No data found, symbol may be delisted
ABR.DE
BNP.PA
HEIA.AS
GEC.DE
- GEC.DE: No data found for this date range, symbol may be delisted
DG.PA
BMW3.DE
ADYEN.AS
BMW.DE
SAF.PA
AXA.DE
CS.PA
DSY.PA
RACE.MI
SHL.DE
REN.AS
BN.PA
RI.PA
PHIA.AS
HEN3.DE
HEN.DE
MUV2.DE
VNA.DE
VODI.F
ISP.MI
IFX.DE
IES.DE
BY6.F
SAN.MC
skipping SAN.MC
BSD2.DE
EN

In [5]:
undervalued

[{'ticker': 'PRX.AS',
  'forcast_price': 1107.7627441615716,
  'current_price': 66.6500015258789,
  'DDM': 6.39},
 {'ticker': 'ASML.AS',
  'forcast_price': 867.8947011650886,
  'current_price': 421.5,
  'DDM': 70.75},
 {'ticker': 'VOW.DE',
  'forcast_price': 552.801798145462,
  'current_price': 169.6999969482422,
  'DDM': 0},
 {'ticker': 'VOW3.DE',
  'forcast_price': 791.0878984438729,
  'current_price': 121.08000183105469,
  'DDM': 0},
 {'ticker': 'DTE.DE',
  'forcast_price': 32.512850719882074,
  'current_price': 19.073999404907227,
  'DDM': 0},
 {'ticker': 'HBC1.DE',
  'forcast_price': 23.00273297902713,
  'current_price': 6.035999774932861,
  'DDM': 0},
 {'ticker': '3CP.F',
  'forcast_price': 12.571701626257978,
  'current_price': 1.639799952507019,
  'DDM': 0},
 {'ticker': 'BPE5.DE',
  'forcast_price': 46.835897337058356,
  'current_price': 4.2845001220703125,
  'DDM': 0},
 {'ticker': 'BPE5.F',
  'forcast_price': 46.835897337058356,
  'current_price': 4.302499771118164,
  'DDM': 0

In [6]:
undervalued_ddm

[{'ticker': 'UNA.AS',
  'forcast_price': 8.297566012354231,
  'current_price': 45.26499938964844,
  'DDM': 88.92},
 {'ticker': 'SAN.PA',
  'forcast_price': 19.504065367278432,
  'current_price': 97.9000015258789,
  'DDM': 186.03}]